In [1]:
import pandas as pd
from sqlalchemy import create_engine
from functools import reduce
import numpy as np
import datetime

In [2]:
deaths_file = "Resources/TeslaAutoDeathsData-tesla-deaths.csv"
tesla_deaths_df = pd.read_csv(deaths_file)
tesla_deaths_df.head()

,year,date,country,state,description,deaths,tesla_driver,tesla_occupant,other_vehicle,cyclists_peds,...,model,autopilot_claimed,verified_tesla_autopilot_death,source,fars_nhtsa_case_filing,note,deceased_1,deceased_2,deceased_3,deceased_4
0,2020.0,2020-10-07,Belgium,NaN,"Driver going wrong direction hits Tesla, drive...",1,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,https://www.sudinfo.be/id263566/article/2020-1...,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.0,2020-09-17,USA,GA,Tesla loses control and crashes into bus shelt...,1,0.0,0.0,0.0,1.0,...,3,1.0,0.0,https://www.ajc.com/news/1-dead-after-tesla-dr...,NaN,NaN,NaN,NaN,NaN,NaN
2,2020.0,2020-09-12,USA,CA,"Car lands on Tesla, other vehicle driver dies",1,0.0,0.0,1.0,0.0,...,NaN,0.0,0.0,https://fox5sandiego.com/news/local-news/1-per...,NaN,NaN,NaN,NaN,NaN,NaN
3,2020.0,2020-09-06,USA,IL,"Tesla runs thru red light, crashes, driver die...",1,1.0,0.0,0.0,0.0,...,NaN,0.0,0.0,https://www.dailyherald.com/news/20200906/driv...,NaN,NaN,NaN,NaN,NaN,NaN
4,2020.0,2020-09-05,China,NaN,"Tesla crashes into crowd, 2 dead and several i...",2,0.0,0.0,0.0,2.0,...,X,0.0,0.0,https://www.globaltimes.cn/content/1200006.shtml,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
stocks_file = "Resources/TSLA-StockDatafrom2010to2020.csv"
tesla_stocks_df = pd.read_csv(stocks_file)
tesla_stocks_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900


In [4]:
tweets_file = "Resources/Tweets2020.csv"
tesla_tweets_df = pd.read_csv(tweets_file)
tesla_tweets_df = tesla_tweets_df.sort_values('date',ascending=True)
tesla_tweets_df

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
11716,11716,15434727182,15434727182,1.275676e+12,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
11715,11715,142179928203460608,142179928203460608,1.322733e+12,2011-12-01 09:55:11,0,NaN,Went to Iceland on Sat to ride bumper cars on ...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
11714,11714,142188458125963264,142188458125963264,1.322735e+12,2011-12-01 10:29:04,0,NaN,I made the volume on the Model S http://t.co/...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
11713,11713,142880871391838208,142880871391838208,1.322900e+12,2011-12-03 08:20:28,0,NaN,"Great Voltaire quote, arguably better than Twa...",en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
11712,11712,142881284019060736,142881284019060736,1.322901e+12,2011-12-03 08:22:07,0,NaN,That was a total non sequitur btw,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,4,1343431408052662273,1343043963096326147,1.609134e+12,2020-12-28 05:39:42,0,NaN,@PPathole @WSJ Absolutely,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'PPathole', 'name': 'Pranay P...",NaN,NaN,NaN,NaN
3,3,1343608530998153222,1343320495127633920,1.609176e+12,2020-12-28 17:23:31,0,NaN,What should Tesla do with in-car gaming in an ...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,2,1343608616960491521,1343576442722893825,1.609176e+12,2020-12-28 17:23:51,0,NaN,@richierichhhhh_ Absolutely,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'richierichhhhh_', 'name': 'R...",NaN,NaN,NaN,NaN
1,1,1343619610617077760,1343386617294295040,1.609179e+12,2020-12-28 18:07:33,0,NaN,@kimpaquette Just meeting with Larry Ellison t...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'kimpaquette', 'name': 'Kim P...",NaN,NaN,NaN,NaN


In [28]:
#Clean the Tweets
tesla_tweets = tesla_tweets_df[['id', 'date', 'tweet', 'username', 'nlikes', 'nreplies', 
                                'nretweets']].reset_index(drop=True)

tesla_tweets['date'] = pd.to_datetime(tesla_tweets['date']).dt.date

#tesla_tweets_dropped = tesla_tweets.dropna()
#Checked for null values. None found

tesla_tweets['date'] = pd.to_datetime(tesla_tweets['date'])

start_date = '01-01-2013'
end_date = '12-31-2020'

mask = (tesla_tweets['date'] > start_date) & (tesla_tweets['date'] <= end_date)

tesla_tweets = tesla_tweets.loc[mask].reset_index(drop=True)

tesla_tweets

,id,date,tweet,username,nlikes,nreplies,nretweets
0,287033019255115777,2013-01-04,This gives a sense of what Mars would look lik...,elonmusk,155,34,291
1,287261992476098560,2013-01-04,Nice story about a Model S coast to coast road...,elonmusk,63,31,147
2,289281686548647936,2013-01-10,"Also, I am not the kale eating overlord of Mar...",elonmusk,58,44,60
3,289423253859082240,2013-01-10,"To be super clear, I don't wish to (nor could ...",elonmusk,85,61,176
4,291332638407528448,2013-01-15,What it feels like to ride a rocket http://t...,elonmusk,119,41,169
...,...,...,...,...,...,...,...
11397,1343431408052662273,2020-12-28,@PPathole @WSJ Absolutely,elonmusk,2394,131,92
11398,1343608530998153222,2020-12-28,What should Tesla do with in-car gaming in an ...,elonmusk,33830,6932,884
11399,1343608616960491521,2020-12-28,@richierichhhhh_ Absolutely,elonmusk,72434,495,907
11400,1343619610617077760,2020-12-28,@kimpaquette Just meeting with Larry Ellison t...,elonmusk,8631,601,314


In [ ]:
#Clean the Stocks
